In [1]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, Input, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
import cv2
import numpy as np
import os
import shutil
import zipfile
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
import requests

In [2]:
import tensorflow_model_optimization as tfmot


In [4]:


# Define a new input layer that accepts grayscale (1-channel) images
input_layer = Input(shape=(224, 224, 1))

# Use Conv2D to change 1-channel input to 3-channel so it can work with MobileNetV2
x = Conv2D(3, (3, 3), padding='same', activation='relu')(input_layer)

# Load pre-trained MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Modify the first layer to accept 1-channel input and broadcast to 3 channels
input_layer = Input(shape=(224, 224, 1))  # Grayscale input
x = Lambda(lambda x: tf.image.grayscale_to_rgb(x))(input_layer)  # Convert grayscale to RGB

# Connect the base model
base_model_output = base_model(x)

# Add custom layers on top of the base model
# (e.g., Global Average Pooling, Dense for classification, etc.)
x = GlobalAveragePooling2D()(base_model_output)
x = Dense(1, activation='sigmoid')(x)  # For binary classification (person vs non-person)

# Create the final model
model = Model(inputs=input_layer, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [5]:
# model = tf.keras.models.load_model('model/model_geeks.h5')
model.load_weights('model/model_geeks.h5')

In [6]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 224, 224, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda_1 (Lambda)               │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 2,225,153 (8.49 MB)

 Non-trainable params: 34,112 (133.25 KB)

In [7]:
# Convert the model to a TensorFlow Lite model with integer quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Enable integer quantization (use representative dataset if available)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# If you have a representative dataset, use it here (optional but recommended)
# def representative_dataset_gen():
#     for input_value in dataset.take(100):
#         yield [input_value]

# converter.representative_dataset = representative_dataset_gen

# Convert the model
tflite_quantized_model = converter.convert()

# Save the converted model
with open('model_quantized.tflite', 'wb') as f:
    f.write(tflite_quantized_model)


INFO:tensorflow:Assets written to: /var/folders/k8/3swkzmkd0p5fyj48wzvwgq6h0000gn/T/tmpgamf1por/assets


INFO:tensorflow:Assets written to: /var/folders/k8/3swkzmkd0p5fyj48wzvwgq6h0000gn/T/tmpgamf1por/assets


Saved artifact at '/var/folders/k8/3swkzmkd0p5fyj48wzvwgq6h0000gn/T/tmpgamf1por'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 1), dtype=tf.float32, name='keras_tensor_156')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  6077308688: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6077310800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6077310224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6077309072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6077310608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6077311376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6077309648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6077310032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6077311760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6077311952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6077313680: Tens

W0000 00:00:1728280068.557511 5902617 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1728280068.558573 5902617 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-10-07 13:47:48.559868: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/k8/3swkzmkd0p5fyj48wzvwgq6h0000gn/T/tmpgamf1por
2024-10-07 13:47:48.565766: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-10-07 13:47:48.565773: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/k8/3swkzmkd0p5fyj48wzvwgq6h0000gn/T/tmpgamf1por
2024-10-07 13:47:48.632936: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-10-07 13:47:48.646160: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-10-07 13:47:49.112440: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at 

In [10]:
import os
file_size = os.path.getsize('yolo_coco.tflite') / (1024 * 1024)
print(f"Quantized TFLite Model Size: {file_size:.2f} MB")


Quantized TFLite Model Size: 0.54 MB


Pruning and Quantize

In [ ]:
import tensorflow as tf
import tensorflow_model_optimization as tfmot

# Load your existing Keras model
model = tf.keras.models.load_model('model/model_geeks.keras')

# Define the pruning parameters
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.50,  # Start with 50% sparsity
        final_sparsity=0.80,    # End with 80% sparsity
        begin_step=0,
        end_step=1000
    )
}

# Apply pruning to the model
pruned_model = tfmot.sparsity.keras.prune_low_magnitude(model, **pruning_params)

# Compile the pruned model
pruned_model.compile(optimizer='adam',
                     loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])

# Train (or fine-tune) the pruned model
pruned_model.fit(train_dataset, validation_data=val_dataset, epochs=5)

# Strip the pruning wrappers for final export
model_for_export = tfmot.sparsity.keras.strip_pruning(pruned_model)

# Quantize and convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_tflite_model = converter.convert()

# Save the quantized and pruned TFLite model
with open('pruned_quantized_model.tflite', 'wb') as f:
    f.write(quantized_tflite_model)

# Check the new size of the model
import os
file_size = os.path.getsize('pruned_quantized_model.tflite') / (1024 * 1024)
print(f"Pruned and Quantized Model Size: {file_size:.2f} MB")
